In [19]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from src import utils_pca

In [2]:
data_source = pd.read_csv("https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-08-03/athletes.csv")
data_raw = data_source
data_raw.head()

,gender,event,medal,athlete,abb,country,grp_id,type,year,guide
0,Men,Double FITA Round Amputee,Gold,LARSEN Finn,DEN,NaN,NaN,Archery,1980,NaN
1,Men,Double FITA Round Amputee,Silver,BRENNE Manfred,FRG,NaN,NaN,Archery,1980,NaN
2,Men,Double FITA Round Amputee,Bronze,SATO Masao,JPN,NaN,NaN,Archery,1980,NaN
3,Men,Double FITA Round Paraplegic,Gold,GEISS H.,FRG,NaN,NaN,Archery,1980,NaN
4,Men,Double FITA Round Paraplegic,Silver,GRUN Guy,BEL,NaN,NaN,Archery,1980,NaN


In [3]:
data_raw

,gender,event,medal,athlete,abb,country,grp_id,type,year,guide
0,Men,Double FITA Round Amputee,Gold,LARSEN Finn,DEN,NaN,NaN,Archery,1980,NaN
1,Men,Double FITA Round Amputee,Silver,BRENNE Manfred,FRG,NaN,NaN,Archery,1980,NaN
2,Men,Double FITA Round Amputee,Bronze,SATO Masao,JPN,NaN,NaN,Archery,1980,NaN
3,Men,Double FITA Round Paraplegic,Gold,GEISS H.,FRG,NaN,NaN,Archery,1980,NaN
4,Men,Double FITA Round Paraplegic,Silver,GRUN Guy,BEL,NaN,NaN,Archery,1980,NaN
...,...,...,...,...,...,...,...,...,...,...
19542,Women,Sitting Volleyball,Bronze,DE ALMEIDA SILVA Nurya,BRA,Brazil,8.0,Volleyball,2016,NaN
19543,Women,Sitting Volleyball,Bronze,DE OLIVEIRA DIAS Edwarda,BRA,Brazil,9.0,Volleyball,2016,NaN
19544,Women,Sitting Volleyball,Bronze,MARIA LEIRIA DE CASTRO Camila,BRA,Brazil,10.0,Volleyball,2016,NaN
19545,Women,Sitting Volleyball,Bronze,PEREIRA Pamela,BRA,Brazil,11.0,Volleyball,2016,NaN


In [7]:
medal_grp = []
grp_id = []
for i in range(19546):
    if data_raw["grp_id"][i]>= 0:
        medal_grp.append(data_raw["medal"][i])
        grp_id.append(data_raw["grp_id"][i])
len(medal_grp)

5118

In [25]:
medal_by_grp_id = pd.DataFrame()
medal_by_grp_id["medal"] = medal_grp
medal_by_grp_id["grp_id"] = grp_id
medal_by_grp_id["grp_id"].unique

<bound method Series.unique of 0        1.0
1        2.0
2        3.0
3        1.0
4        2.0
        ... 
5113     7.0
5114     8.0
5115     9.0
5116    10.0
5117    11.0
Name: grp_id, Length: 5118, dtype: float64>

In [21]:
medal_guide = []
guide = []
def isNaN(string):
    return string != string
for i in range(19546):
    if  isNaN(data_raw["guide"][i]) == False:
        medal_guide.append(data_raw["medal"][i])
        guide.append(data_raw["guide"][i])
len(medal_guide)

53

In [23]:
medal_by_guide = pd.DataFrame()
medal_by_guide["medal"] = medal_guide
medal_by_guide["guilde"] = guide
medal_by_guide

,medal,guilde
0,Gold,AVERY Jerome
1,Silver,SILVA Celio
2,Bronze,TJIVIJU Even
3,Gold,TJIVIJU Even
4,Silver,SILVA Jonas
5,Bronze,DE OLIVEIRA SALES Heitor
6,Gold,BLANQUINO EXPOSITO Marcos
7,Silver,SILVA Jonas
8,Bronze,SHIMANDA Sem
9,Gold,BOIT James
